# Text cleaning

In [5]:
import requests
import time
import tqdm
import pandas as pd
import os
import json
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import re
import nltk

# Importing DR and Berlingske texts and merging them into one

In [ ]:
data_DR = pd.read_csv('DR_full_data.csv')
data_DR['source'] = 'dr.dk' # adding a source variable

data_B = pd.read_csv('Berlingske_full_data.csv')
data_B['source'] = 'berlingske.dk' # adding a source variable

In [ ]:
# !!!! MODIFY !!! 
# Making sure we only keep relevant columns and giving them the same names:
data_CNN = data_CNN[['headline', 'url', 'lastModifiedDate','article_text','source']]
data_CNN = data_CNN.rename(columns={'headline': 'title', 'lastModifiedDate': 'date'})

data_ABC = data_ABC.rename(columns = {'URL':'url', 'Title':'title', 'Date':'date'})

In [ ]:
# !!!! MODIFY !!!
# Convert to date-time format 

# DR also has the time included, which confuses when trying to convert to datetime format, so we will remove those first:
data_DR['date'] = data_DR['date'].str.replace(r'T.*', '', regex=True)
data_DR['date'] = pd.to_datetime(data_DR['date'], errors='coerce')


# Berlingske: check how the datetime is formatted before
data_ABC['date'] = pd.to_datetime(data_ABC['date'], errors='coerce')

In [ ]:
# Merging them into one:
data = pd.concat([data_DR, data_B], axis=0)

In [ ]:
# Sort the DataFrame by the 'article_date' column from oldest to newest
data = data.sort_values(by='date')

# While we wait for the Berlingske articles, i just try with the DR.dk articles: 

In [15]:
## DELETE THIS AFTER BERLINGSKE IS READY 
data = pd.read_csv('DR_full_data.csv')

In [16]:
data['date'] = data['date'].str.replace(r'T.*', '', regex=True)
data['date'] = pd.to_datetime(data['date'], errors='coerce')

## What is your document?

Our document is the article as a whole - all text in the article excluding image discriptions, and authour tags. 

## Preprossesing
- Clean text: ignore/remove any unwanted characters: casing, HTML markup, non-words, etc. (maybe also emoticons?)
- Tokenization and stop-words
- Stemming and lemmatization

***Removing NA values*** on the column "article_text"

In [20]:
# Remove rows with NaN values in the 'article_text' column
data = data.dropna(subset=['article_text'])
data = data.reset_index(drop=True)  # Reset the index

***Cleaning the text***

In [21]:
def cleaner(document):
    document = document.lower()  # To lower case
    document = re.sub(r'<[^>]*>', ' ', document)  # Remove HTML
    document = re.sub(r'[^\w\s&$€%]', '', document)  # Remove non-alphanumeric characters except &, $, %, and €
    return document

data['article_text'] = data['article_text'].apply(cleaner)

# Checking for duplicates: 
data[data['article_text'].duplicated()]

# Removing the duplicates articles: 
data = data.drop_duplicates(subset=['article_text']).reset_index(drop=True)

***Tokenization***
- Splitting the articles into meaningfull elements to prepare for analysis. In our case we need to split the articles into words as these are what will be used for classifying sentiments. 

In [22]:
# Split the 'article_text' column into tokens based on whitespace and saving it into a new column
# "words"
data['words'] = data['article_text'].str.split()

***Removing stop words***
- These are words that occur very often and probably bear no useful information about the text

In [23]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop = nltk.corpus.stopwords.words('danish')
data['words'] = [i for i in data['words'] if i not in stop]

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/astakettel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


***Saving as a CSV file***

In [ ]:
data.to_csv('DK_cleaned.csv', index=False)